In [1]:
!pip install optuna


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report, confusion_matrix
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine


In [3]:
car = pd.read_excel('CarFINALdataset.xlsx')

In [4]:
car.head()

,User_ID,travelCode,Check-in,pickupLocation,dropoffLocation,carType,rentalAgency,rentalDuration,Car_total_distance,fuelPolicy,Car_bookingStatus,total_rent_price
0,0,22,02/27/2020,Johntown,Port Brian,Sedan,Sixt,3,285,Prepaid,Pending,3410
1,0,45,08/06/2020,Owensland,Ruizfort,Sedan,Enterprise,3,412,Full-to-Full,Pending,4644
2,2,117,02/27/2020,Edwardview,Katiefort,Sedan,Hertz,1,433,Partial,Confirmed,4740
3,2,119,03/12/2020,Moranborough,Lake Stephanie,Hatchback,Budget,3,421,Full-to-Full,Confirmed,3744
4,2,153,11/05/2020,Port Kathrynstad,East Ronnieberg,Sedan,Enterprise,5,100,Prepaid,Cancelled,1694


In [5]:
car

,User_ID,travelCode,Check-in,pickupLocation,dropoffLocation,carType,rentalAgency,rentalDuration,Car_total_distance,fuelPolicy,Car_bookingStatus,total_rent_price
0,0,22,02/27/2020,Johntown,Port Brian,Sedan,Sixt,3,285,Prepaid,Pending,3410
1,0,45,08/06/2020,Owensland,Ruizfort,Sedan,Enterprise,3,412,Full-to-Full,Pending,4644
2,2,117,02/27/2020,Edwardview,Katiefort,Sedan,Hertz,1,433,Partial,Confirmed,4740
3,2,119,03/12/2020,Moranborough,Lake Stephanie,Hatchback,Budget,3,421,Full-to-Full,Confirmed,3744
4,2,153,11/05/2020,Port Kathrynstad,East Ronnieberg,Sedan,Enterprise,5,100,Prepaid,Cancelled,1694
...,...,...,...,...,...,...,...,...,...,...,...,...
3768,1335,135831,12/29/2022,West Donna,Fordshire,Sedan,Enterprise,5,123,Prepaid,Cancelled,1898
3769,1335,135843,03/23/2023,East Crystalview,Lake Laura,Sedan,Hertz,2,229,Full-to-Full,Confirmed,2743
3770,1337,135892,04/16/2020,Kylehaven,Johnland,Luxury,Enterprise,1,285,Prepaid,Cancelled,6692
3771,1339,135911,12/12/2019,South Julie,Brentside,Sedan,Sixt,1,155,Full-to-Full,Pending,1952


In [6]:
passenger = pd.read_excel('PassengerFINALdataset.xlsx')

In [7]:
passenger.head()

,User_ID,company,Name,gender_x
0,0,4You,Roy Braun,male
1,1,4You,Joseph Holsten,male
2,2,4You,Wilma Mcinnis,female
3,3,4You,Paula Daniel,female
4,4,4You,Patricia Carson,female


In [9]:
car = pd.merge(car,passenger,how='left',on='User_ID')

In [10]:
car.head()

,User_ID,travelCode,Check-in,pickupLocation,dropoffLocation,carType,rentalAgency,rentalDuration,Car_total_distance,fuelPolicy,Car_bookingStatus,total_rent_price,company,Name,gender_x
0,0,22,02/27/2020,Johntown,Port Brian,Sedan,Sixt,3,285,Prepaid,Pending,3410,4You,Roy Braun,male
1,0,45,08/06/2020,Owensland,Ruizfort,Sedan,Enterprise,3,412,Full-to-Full,Pending,4644,4You,Roy Braun,male
2,2,117,02/27/2020,Edwardview,Katiefort,Sedan,Hertz,1,433,Partial,Confirmed,4740,4You,Wilma Mcinnis,female
3,2,119,03/12/2020,Moranborough,Lake Stephanie,Hatchback,Budget,3,421,Full-to-Full,Confirmed,3744,4You,Wilma Mcinnis,female
4,2,153,11/05/2020,Port Kathrynstad,East Ronnieberg,Sedan,Enterprise,5,100,Prepaid,Cancelled,1694,4You,Wilma Mcinnis,female


In [11]:
car.drop(['Name','User_ID','travelCode','company'],inplace=True,axis=1)

In [12]:
car

,Check-in,pickupLocation,dropoffLocation,carType,rentalAgency,rentalDuration,Car_total_distance,fuelPolicy,Car_bookingStatus,total_rent_price,gender_x
0,02/27/2020,Johntown,Port Brian,Sedan,Sixt,3,285,Prepaid,Pending,3410,male
1,08/06/2020,Owensland,Ruizfort,Sedan,Enterprise,3,412,Full-to-Full,Pending,4644,male
2,02/27/2020,Edwardview,Katiefort,Sedan,Hertz,1,433,Partial,Confirmed,4740,female
3,03/12/2020,Moranborough,Lake Stephanie,Hatchback,Budget,3,421,Full-to-Full,Confirmed,3744,female
4,11/05/2020,Port Kathrynstad,East Ronnieberg,Sedan,Enterprise,5,100,Prepaid,Cancelled,1694,female
...,...,...,...,...,...,...,...,...,...,...,...
3768,12/29/2022,West Donna,Fordshire,Sedan,Enterprise,5,123,Prepaid,Cancelled,1898,male
3769,03/23/2023,East Crystalview,Lake Laura,Sedan,Hertz,2,229,Full-to-Full,Confirmed,2743,male
3770,04/16/2020,Kylehaven,Johnland,Luxury,Enterprise,1,285,Prepaid,Cancelled,6692,male
3771,12/12/2019,South Julie,Brentside,Sedan,Sixt,1,155,Full-to-Full,Pending,1952,male


In [13]:
car['Car_check'] = pd.to_datetime(car['Check-in'])

In [15]:
car.drop('Check-in',inplace=True,axis=1)

In [16]:
car['Weekend_Checkin'] = (car['Car_check'].dt.weekday >= 5).astype(int)

In [17]:
car['Month_checkin'] = car['Car_check'].dt.month

In [18]:
car.head()

,pickupLocation,dropoffLocation,carType,rentalAgency,rentalDuration,Car_total_distance,fuelPolicy,Car_bookingStatus,total_rent_price,gender_x,Car_check,Weekend_Checkin,Month_checkin
0,Johntown,Port Brian,Sedan,Sixt,3,285,Prepaid,Pending,3410,male,2020-02-27,0,2
1,Owensland,Ruizfort,Sedan,Enterprise,3,412,Full-to-Full,Pending,4644,male,2020-08-06,0,8
2,Edwardview,Katiefort,Sedan,Hertz,1,433,Partial,Confirmed,4740,female,2020-02-27,0,2
3,Moranborough,Lake Stephanie,Hatchback,Budget,3,421,Full-to-Full,Confirmed,3744,female,2020-03-12,0,3
4,Port Kathrynstad,East Ronnieberg,Sedan,Enterprise,5,100,Prepaid,Cancelled,1694,female,2020-11-05,0,11


In [19]:
car1 = car.copy(deep=True)

In [20]:
car1

,pickupLocation,dropoffLocation,carType,rentalAgency,rentalDuration,Car_total_distance,fuelPolicy,Car_bookingStatus,total_rent_price,gender_x,Car_check,Weekend_Checkin,Month_checkin
0,Johntown,Port Brian,Sedan,Sixt,3,285,Prepaid,Pending,3410,male,2020-02-27,0,2
1,Owensland,Ruizfort,Sedan,Enterprise,3,412,Full-to-Full,Pending,4644,male,2020-08-06,0,8
2,Edwardview,Katiefort,Sedan,Hertz,1,433,Partial,Confirmed,4740,female,2020-02-27,0,2
3,Moranborough,Lake Stephanie,Hatchback,Budget,3,421,Full-to-Full,Confirmed,3744,female,2020-03-12,0,3
4,Port Kathrynstad,East Ronnieberg,Sedan,Enterprise,5,100,Prepaid,Cancelled,1694,female,2020-11-05,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3768,West Donna,Fordshire,Sedan,Enterprise,5,123,Prepaid,Cancelled,1898,male,2022-12-29,0,12
3769,East Crystalview,Lake Laura,Sedan,Hertz,2,229,Full-to-Full,Confirmed,2743,male,2023-03-23,0,3
3770,Kylehaven,Johnland,Luxury,Enterprise,1,285,Prepaid,Cancelled,6692,male,2020-04-16,0,4
3771,South Julie,Brentside,Sedan,Sixt,1,155,Full-to-Full,Pending,1952,male,2019-12-12,0,12


In [44]:
X = car.drop('total_rent_price',axis=1)
y = car['total_rent_price']

In [45]:
num_feature = X.select_dtypes(include=['int64','float64']).columns
cat_feature = X.select_dtypes(include=['object']).columns

In [46]:
num_transformer = Pipeline([('imputer',SimpleImputer(strategy='median')),
                           ('scaler',StandardScaler())])

In [47]:
cat_transformer = Pipeline([('imputer',SimpleImputer(strategy='most_frequent')),
                           ('onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False))])

In [48]:
preprocessor = ColumnTransformer([
    ('num',num_transformer,num_feature),
    ('cat',cat_transformer,cat_feature)
])

In [49]:
X_transformed = preprocessor.fit_transform(X)

In [62]:
feature_names = X.columns

In [63]:
X_train,X_test,y_train,y_test = train_test_split(X_transformed,y,test_size=0.2,random_state=42)

In [29]:
def objective(trial):
    model = XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 100, 500),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2),
        max_depth=trial.suggest_int('max_depth', 3, 10),
        subsample=trial.suggest_float('subsample', 0.5, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.5, 1.0),
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred)


In [30]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
best_params = study.best_params


[I 2025-02-17 17:21:01,750] A new study created in memory with name: no-name-2f3ffb1f-b9a6-4a2e-af70-af195f0102de
[I 2025-02-17 17:21:28,932] Trial 0 finished with value: 7662.91317734897 and parameters: {'n_estimators': 298, 'learning_rate': 0.024819110624814483, 'max_depth': 9, 'subsample': 0.6186081171954754, 'colsample_bytree': 0.7870590392413841}. Best is trial 0 with value: 7662.91317734897.
[I 2025-02-17 17:21:44,709] Trial 1 finished with value: 10670.259721459719 and parameters: {'n_estimators': 223, 'learning_rate': 0.07979066361869441, 'max_depth': 9, 'subsample': 0.5785289755103764, 'colsample_bytree': 0.7502154362606421}. Best is trial 0 with value: 7662.91317734897.
[I 2025-02-17 17:21:57,762] Trial 2 finished with value: 2118.6389690400206 and parameters: {'n_estimators': 479, 'learning_rate': 0.12843665390594952, 'max_depth': 4, 'subsample': 0.6180954121411721, 'colsample_bytree': 0.6375038389774774}. Best is trial 2 with value: 2118.6389690400206.
[I 2025-02-17 17:22:0

[I 2025-02-17 17:29:01,240] Trial 28 finished with value: 965.3643333695484 and parameters: {'n_estimators': 420, 'learning_rate': 0.06750619503361265, 'max_depth': 6, 'subsample': 0.7156642253313118, 'colsample_bytree': 0.9203754130341333}. Best is trial 28 with value: 965.3643333695484.
[I 2025-02-17 17:29:19,695] Trial 29 finished with value: 2140.010120145039 and parameters: {'n_estimators': 419, 'learning_rate': 0.031271742586304385, 'max_depth': 5, 'subsample': 0.7544431495854348, 'colsample_bytree': 0.7656332711972351}. Best is trial 28 with value: 965.3643333695484.


In [31]:
best_xgb = XGBRegressor(**best_params)
best_xgb.fit(X_train, y_train)
y_pred_xgb = best_xgb.predict(X_test)

In [32]:
stacked_model = StackingRegressor(
    estimators=[('xgb', best_xgb)],
    final_estimator=XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=3, random_state=42)
)


In [33]:
stacked_model.fit(X_train, y_train)
y_pred_stack = stacked_model.predict(X_test)


In [34]:
def build_nn():
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae'])
    return model

nn_model = build_nn()
nn_callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]
nn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=1, callbacks=nn_callbacks)

y_pred_nn = nn_model.predict(X_test).flatten()


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - loss: 20931218.0000 - mae: 3981.3125 - val_loss: 1234057.1250 - val_mae: 843.7166 - learning_rate: 0.0100
Epoch 2/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 1920023.5000 - mae: 1108.3278 - val_loss: 1161221.6250 - val_mae: 973.7256 - learning_rate: 0.0100
Epoch 3/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 917589.6250 - mae: 746.7968 - val_loss: 158101.0312 - val_mae: 321.3967 - learning_rate: 0.0100
Epoch 4/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 727435.1875 - mae: 630.9664 - val_loss: 336486.2812 - val_mae: 443.4665 - learning_rate: 0.0100
Epoch 5/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 597226.3125 - mae: 581.2196 - val_loss: 73463.5547 - val_mae: 219.7439 - learning_rate: 0.0100
Epoch 6/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 538436.8125 - mae: 555.3964 - val_loss: 55788.9922 - val_mae: 178.3197 - learning_rate: 0.0100
Epoch 7/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss:

In [35]:
def evaluate_model(name, y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # classification__report = classification_report(y_true,y_pred)
    # confusion__matrix = confusion_matrix(y_true,y_pred)
    print(f"\n {name} Performance:")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    
evaluate_model("Optimized XGBoost", y_test, y_pred_xgb)
evaluate_model("Stacking Model", y_test, y_pred_stack)
evaluate_model("Neural Network", y_test, y_pred_nn)


 Optimized XGBoost Performance:
R² Score: 0.9998
MAE: 27.0805
RMSE: 35.9514

 Stacking Model Performance:
R² Score: 0.9996
MAE: 34.8109
RMSE: 48.3622

 Neural Network Performance:
R² Score: 0.9943
MAE: 135.4522
RMSE: 179.3486


In [36]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU, Add, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import StandardScaler
import numpy as np



def build_nn():
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(128, kernel_regularizer=l2(0.001))(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.2)(x)

    x1 = Dense(64, kernel_regularizer=l2(0.001))(x)
    x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    x1 = Dropout(0.2)(x1)

    x2 = Dense(128, kernel_regularizer=l2(0.001))(x1)  # Ensuring same shape as `x`
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU()(x2)
    x2 = Dropout(0.2)(x2)

    # Residual connection (now both are (128,))
    x3 = Add()([x, x2])
    
    outputs = Dense(1)(x3)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=AdamW(learning_rate=0.005), loss='mse', metrics=['mae'])
    return model

nn_model = build_nn()
nn_callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

nn_model.fit(X_train, y_train, validation_data=(X_test, y_test), 
             epochs=200, batch_size=32, verbose=1, callbacks=nn_callbacks)

y_pred_nn = nn_model.predict(X_test).flatten()


Epoch 1/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - loss: 22948020.0000 - mae: 4165.2476 - val_loss: 22248468.0000 - val_mae: 4135.4673 - learning_rate: 0.0050
Epoch 2/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 19880568.0000 - mae: 3934.7280 - val_loss: 16671508.0000 - val_mae: 3671.4983 - learning_rate: 0.0050
Epoch 3/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 16054231.0000 - mae: 3621.5251 - val_loss: 10213079.0000 - val_mae: 3004.3381 - learning_rate: 0.0050
Epoch 4/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 10794491.0000 - mae: 3044.4666 - val_loss: 5665068.5000 - val_mae: 2314.1086 - learning_rate: 0.0050
Epoch 5/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 6823207.0000 - mae: 2461.3994 - val_loss: 3280339.0000 - val_mae: 1760.7616 - learning_rate: 0.0050
Epoch 6/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 3915817.5000 - mae: 1847.9645 - val_loss: 1726540.0000 - val_mae: 1234.0496 - learning_rate: 0.0050
Epoch 7/200
95/95 ━━━━━━━━━━━━━━━

In [37]:
evaluate_model("Neural Network", y_test, y_pred_nn)


 Neural Network Performance:
R² Score: 0.9701
MAE: 310.9343
RMSE: 412.4315


In [38]:
!pip install scikeras

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
from sklearn.ensemble import StackingRegressor
from scikeras.wrappers import KerasRegressor  # Corrected import

def build_nn_wrapper():
    return build_nn()

nn_wrapper = KerasRegressor(build_fn=build_nn_wrapper, epochs=100, batch_size=32, verbose=0)

stacked_model = StackingRegressor(
    estimators=[('xgb', best_xgb), ('nn', nn_wrapper)],
    final_estimator=XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=3, random_state=42)
)

stacked_model.fit(X_train, y_train)
y_pred_stacked = stacked_model.predict(X_test)


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X,

In [40]:
evaluate_model("Sta", y_test, y_pred_stacked)


 Sta Performance:
R² Score: 0.9996
MAE: 34.6844
RMSE: 48.1538


In [41]:
import numpy as np
X_train = np.log1p(X_train)
X_test = np.log1p(X_test)
y_train = np.log1p(y_train)
y_test = np.log1p(y_test)


C:\Users\hp\AppData\Local\Temp\ipykernel_1028\1480503297.py:2: RuntimeWarning: invalid value encountered in log1p
  X_train = np.log1p(X_train)
C:\Users\hp\AppData\Local\Temp\ipykernel_1028\1480503297.py:3: RuntimeWarning: invalid value encountered in log1p
  X_test = np.log1p(X_test)


In [55]:
import numpy as np
print(np.isnan(X_train).sum())  # Check the number of NaN values in X_train
print(np.isnan(X_test).sum())   # Check the number of NaN values in X_test


0
0


In [56]:
print((X_train < -1).sum())  # Checking for values less than -1 in X_train
print((X_test < -1).sum())   # Checking for values less than -1 in X_test


1830
444


In [64]:
import numpy as np

# Identify columns with negative values in X_train
negative_columns_train = np.any(X_train < -1, axis=0)
print("Columns in X_train with negative values:")
print(np.array(feature_names)[negative_columns_train])  # feature_names should be your original column names

# Identify columns with negative values in X_test
negative_columns_test = np.any(X_test < -1, axis=0)
print("Columns in X_test with negative values:")
print(np.array(feature_names)[negative_columns_test])



Columns in X_train with negative values:


IndexError: boolean index did not match indexed array along dimension 0; dimension is 12 but corresponding boolean dimension is 6703

In [67]:
X_train

array([[ 0.72286285,  1.53677057,  0.13450465, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.02101994, -0.50936861, -0.70155141, ...,  0.        ,
         0.        ,  1.        ],
       [-1.38266586, -1.57089946, -0.42286606, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.68082296, -0.54782987, -1.25892212, ...,  1.        ,
         0.        ,  0.        ],
       [-1.38266586, -1.52474595, -1.25892212, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.72286285, -1.27859387, -0.70155141, ...,  1.        ,
         0.        ,  0.        ]])

In [69]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

def objective(trial):
    model = XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 100, 800),
        learning_rate=trial.suggest_float('learning_rate', 0.005, 0.2),
        max_depth=trial.suggest_int('max_depth', 3, 12),
        subsample=trial.suggest_float('subsample', 0.6, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.6, 1.0),
        reg_lambda=trial.suggest_float('reg_lambda', 0.1, 10.0),
        reg_alpha=trial.suggest_float('reg_alpha', 0.1, 10.0),
        gamma=trial.suggest_float('gamma', 0, 5),
        min_child_weight=trial.suggest_int('min_child_weight', 1, 10),
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
best_params = study.best_params

best_xgb = XGBRegressor(**best_params)
best_xgb.fit(X_train, y_train)
y_pred_xgb = best_xgb.predict(X_test)


[I 2025-02-18 16:52:59,261] A new study created in memory with name: no-name-fcc52fdf-0712-4d51-a86c-2d38d17eaa76
[I 2025-02-18 16:53:34,674] Trial 0 finished with value: 1512.5529726697957 and parameters: {'n_estimators': 388, 'learning_rate': 0.025457099820793718, 'max_depth': 12, 'subsample': 0.8053194498793359, 'colsample_bytree': 0.9057054133998108, 'reg_lambda': 9.445907107486036, 'reg_alpha': 3.641517632338189, 'gamma': 4.147365363125369, 'min_child_weight': 8}. Best is trial 0 with value: 1512.5529726697957.
[I 2025-02-18 16:53:40,779] Trial 1 finished with value: 3465.6356297329166 and parameters: {'n_estimators': 201, 'learning_rate': 0.19035164741046257, 'max_depth': 4, 'subsample': 0.906849221558546, 'colsample_bytree': 0.7771720318008701, 'reg_lambda': 1.338179037494193, 'reg_alpha': 1.7948380212327297, 'gamma': 3.792087386958186, 'min_child_weight': 1}. Best is trial 0 with value: 1512.5529726697957.
[I 2025-02-18 16:54:02,986] Trial 2 finished with value: 17315.113279949

[I 2025-02-18 17:03:31,680] Trial 20 finished with value: 891.8804016576994 and parameters: {'n_estimators': 692, 'learning_rate': 0.0374781360721899, 'max_depth': 6, 'subsample': 0.6055023001384049, 'colsample_bytree': 0.9446255019395262, 'reg_lambda': 2.6316249936148735, 'reg_alpha': 9.022617531832548, 'gamma': 1.3684746888704715, 'min_child_weight': 7}. Best is trial 20 with value: 891.8804016576994.
[I 2025-02-18 17:04:12,064] Trial 21 finished with value: 882.517844965304 and parameters: {'n_estimators': 799, 'learning_rate': 0.0334434958442328, 'max_depth': 6, 'subsample': 0.620259516675092, 'colsample_bytree': 0.9526780465359408, 'reg_lambda': 2.4608612265839325, 'reg_alpha': 9.03497284848793, 'gamma': 1.2200119627043635, 'min_child_weight': 7}. Best is trial 21 with value: 882.517844965304.
[I 2025-02-18 17:05:11,296] Trial 22 finished with value: 1135.3121193554869 and parameters: {'n_estimators': 787, 'learning_rate': 0.0312409575725196, 'max_depth': 8, 'subsample': 0.6019909

[I 2025-02-18 17:15:55,563] Trial 41 finished with value: 949.3787892181007 and parameters: {'n_estimators': 765, 'learning_rate': 0.04022082506302523, 'max_depth': 6, 'subsample': 0.7390709729717787, 'colsample_bytree': 0.9690373878204449, 'reg_lambda': 2.249173735567502, 'reg_alpha': 9.728054025764985, 'gamma': 3.5228624272272877, 'min_child_weight': 9}. Best is trial 21 with value: 882.517844965304.
[I 2025-02-18 17:16:32,004] Trial 42 finished with value: 938.0903405966349 and parameters: {'n_estimators': 766, 'learning_rate': 0.037285229228060976, 'max_depth': 6, 'subsample': 0.7045887089449334, 'colsample_bytree': 0.9688745344479339, 'reg_lambda': 2.039159267808618, 'reg_alpha': 9.507650309133826, 'gamma': 3.7416582541351047, 'min_child_weight': 9}. Best is trial 21 with value: 882.517844965304.
[I 2025-02-18 17:17:13,171] Trial 43 finished with value: 1816.133991566853 and parameters: {'n_estimators': 706, 'learning_rate': 0.19961271460559482, 'max_depth': 7, 'subsample': 0.6176

In [70]:
evaluate_model("New_XGB", y_test, y_pred_xgb)


 New_XGB Performance:
R² Score: 0.9998
MAE: 23.1816
RMSE: 31.3595


In [71]:
y_over = best_xgb.predict(X_train)

In [72]:
evaluate_model("New_XGB", y_train, y_over)


 New_XGB Performance:
R² Score: 1.0000
MAE: 10.5978
RMSE: 13.6961
